In [1]:
import sqlite3
import pandas as pd 
conn = sqlite3.connect('data_input/loan2.db')
loans = pd.read_sql_query('SELECT * FROM record', conn)
loans.head()

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077501,2011,01/12/2011,1012015,10.0,RENT,1,Low,24000,1,...,0,10.65,B,2,27.65,5861.071414,5000.0,0.0,162.87,munster
1,1077430,2011,01/12/2011,1042013,0.5,RENT,1,Low,30000,1,...,1,15.27,C,3,1.0,1008.71,456.46,117.08,59.83,leinster
2,1077175,2011,01/12/2011,1062014,10.0,RENT,1,Low,12252,1,...,0,15.96,C,3,8.72,3003.653644,2400.0,0.0,84.33,cannught
3,1076863,2011,01/12/2011,1012015,10.0,RENT,1,Low,49200,1,...,0,13.49,C,3,20.0,12226.30221,10000.0,0.0,339.31,ulster
4,1075358,2011,01/12/2011,1012016,1.0,RENT,1,Low,80000,1,...,0,12.69,B,2,17.94,3242.17,2233.1,0.0,67.79,ulster


In [2]:
import os
print(os.getcwd())

/home/abhinowo/Documents/LearningProject/PythonDataAnalytics/bankersSeries


In [3]:
df = loans.copy()

In [4]:
df.columns

Index(['id', 'year', 'issue_d', 'final_d', 'emp_length_int', 'home_ownership',
       'home_ownership_cat', 'income_category', 'annual_inc', 'income_cat',
       'loan_amount', 'term', 'term_cat', 'application_type',
       'application_type_cat', 'purpose', 'purpose_cat', 'interest_payments',
       'interest_payment_cat', 'loan_condition', 'loan_condition_cat',
       'interest_rate', 'grade', 'grade_cat', 'dti', 'total_pymnt',
       'total_rec_prncp', 'recoveries', 'installment', 'region'],
      dtype='object')

In [6]:
# df.groupby(['home_ownership','income_category']).sum().sort_values(by='annual_inc', ascending=False).head()

In [9]:
top_region = pd.read_sql_query("SELECT id, region as Negara, home_ownership as Tempat, SUM(loan_amount) as Debt \
                               FROM record \
                               GROUP BY region \
                               ORDER BY SUM(loan_amount) \
                               LIMIT 5", con=conn, index_col='id')

top_region

,Negara,Tempat,Debt
id,,,
1077501,munster,RENT,1592905675
1077175,cannught,RENT,2255721475
1069971,Northern-Irl,MORTGAGE,3051049025
1076863,ulster,RENT,3055134900
1077430,leinster,RENT,3138700875


In [20]:
debt_consol = pd.read_sql_query("""
    SELECT issue_d, region, purpose
    FROM record
    WHERE purpose = 'debt_consolidation'
    ORDER BY purpose
    LIMIT 10;
""", con=conn)

debt_consol

,issue_d,region,purpose
0,01/12/2011,leinster,debt_consolidation
1,01/12/2011,munster,debt_consolidation
2,01/12/2011,ulster,debt_consolidation
3,01/12/2011,leinster,debt_consolidation
4,01/12/2011,cannught,debt_consolidation
5,01/12/2011,ulster,debt_consolidation
6,01/12/2011,munster,debt_consolidation
7,01/12/2011,leinster,debt_consolidation
8,01/12/2011,ulster,debt_consolidation
9,01/12/2011,Northern-Irl,debt_consolidation


In [24]:
bad_loan = pd.read_sql_query("""
    SELECT issue_d, purpose, grade, loan_amount
    FROM record
    WHERE grade NOT IN ('A', 'B') OR loan_condition = 'Bad Loan'
    ORDER BY loan_amount DESC
    LIMIT 10;
""", con=conn)

bad_loan

,issue_d,purpose,grade,loan_amount
0,01/11/2011,car,D,9975
1,01/09/2011,debt_consolidation,D,9975
2,01/05/2011,other,C,9975
3,01/04/2011,debt_consolidation,C,9975
4,01/02/2011,other,C,9975
5,01/04/2010,medical,C,9975
6,01/01/2010,medical,B,9975
7,01/11/2008,debt_consolidation,D,9975
8,01/04/2009,home_improvement,E,9975
9,01/11/2013,credit_card,C,9975


In [25]:
pd.read_sql_query("""
    SELECT DISTINCT purpose
    from record
""", con=conn)

,purpose
0,credit_card
1,car
2,small_business
3,other
4,wedding
5,debt_consolidation
6,home_improvement
7,major_purchase
8,medical
9,moving


In [26]:
pd.read_sql_query("""
    SELECT DISTINCT region
    from record
""", con=conn)

,region
0,munster
1,leinster
2,cannught
3,ulster
4,Northern-Irl


### Like

 SQLite provides two wildcards for constructing patterns. They are percent sign % and underscore _ :

- `%` wildcard matches any sequence of zero or more characters.
- `_` wildcard matches any single character.
- `b%` matches `bank`, `bytes`, `bit`
- `%ion` matches `institution`, `renovation`
- `%it% matches any string containing `it`

- `b_nk` matches `bank`, `bunk`
- `l__n` matches `loan`, `lean`

- "A" LIKE "a" is true (non case sensitive)
- to set case-sensitive: PRAGMA case_sensitive_like = true;

In [27]:
pd.read_sql_query("""
    SELECT issue_d, region, purpose
    FROM record
    WHERE purpose LIKE '%business'
    ORDER BY purpose
    LIMIT 10;             
""", con=conn)

,issue_d,region,purpose
0,01/12/2011,cannught,small_business
1,01/12/2011,ulster,small_business
2,01/12/2011,Northern-Irl,small_business
3,01/12/2011,ulster,small_business
4,01/12/2011,ulster,small_business
5,01/12/2011,munster,small_business
6,01/12/2011,Northern-Irl,small_business
7,01/12/2011,leinster,small_business
8,01/12/2011,ulster,small_business
9,01/12/2011,ulster,small_business


In [29]:
pd.read_sql_query("""
    SELECT * 
    FROM record
    WHERE 
        purpose LIKE '%business'
        AND term like '%36%'
    ORDER BY purpose
    LIMIT 10              
""", con=conn)

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077175,2011,01/12/2011,1062014,10.0,RENT,1,Low,12252,1,...,0,15.96,C,3,8.72,3003.653644,2400.0,0.0,84.33,cannught
1,1051117,2011,01/12/2011,1012015,6.0,RENT,1,Low,80000,1,...,0,9.91,B,2,10.0,16239.914130000001,14000.0,0.0,451.15,Northern-Irl
2,1068997,2011,01/12/2011,1082012,1.0,MORTGAGE,3,Low,52000,1,...,0,7.9,A,1,9.65,15557.64,15000.0,0.0,469.36,munster
3,1068273,2011,01/12/2011,1012015,8.0,MORTGAGE,3,Medium,110000,2,...,0,6.62,A,1,9.69,6077.8756189999995,5500.0,0.0,168.88,Northern-Irl
4,1060662,2011,01/12/2011,1012015,5.0,MORTGAGE,3,Low,26500,1,...,0,6.03,A,1,18.2,6793.070094,6200.0,0.0,188.71,leinster
5,1067419,2011,01/12/2011,1092014,2.0,RENT,1,Low,45900,1,...,0,13.49,C,3,24.97,12174.48231,10000.0,0.0,339.31,ulster
6,1067364,2011,01/12/2011,1032014,5.0,RENT,1,Low,65000,1,...,1,13.49,C,3,6.92,9270.84,6799.56,459.53,339.31,ulster
7,1066081,2011,01/12/2011,1102012,10.0,RENT,1,Low,18000,1,...,0,6.62,A,1,23.8,1880.09,1800.0,0.0,55.27,cannught
8,1063778,2011,01/12/2011,1022012,1.0,RENT,1,Medium,110000,2,...,1,16.29,D,4,21.99,1873.45,655.77,815.11,529.51,munster
9,1056914,2011,01/12/2011,1112014,3.0,RENT,1,Low,80000,1,...,0,17.27,D,4,5.73,12874.2358,10000.0,0.0,357.88,Northern-Irl


### Groupby

In [30]:
pd.read_sql_query("""
    SELECT purpose, COUNT(purpose) as Frequency
    from record
    GROUP BY purpose
    ORDER BY frequency DESC                  
""", con=conn)

,purpose,Frequency
0,debt_consolidation,524215
1,credit_card,206182
2,home_improvement,51829
3,other,42894
4,major_purchase,17277
5,small_business,10377
6,car,8863
7,medical,8540
8,moving,5414
9,vacation,4736


In [32]:
pd.read_sql_query("""
    SELECT purpose, AVG(loan_amount) as AvgAmount, term
    from record
    GROUP BY purpose, term
    ORDER BY term                  
""", con=conn)

,purpose,AvgAmount,term
0,car,7729.213402,36 months
1,credit_card,13401.627884,36 months
2,debt_consolidation,13064.812291,36 months
3,educational,6618.719212,36 months
4,home_improvement,11735.794820,36 months
5,house,12415.755258,36 months
6,major_purchase,9352.416756,36 months
7,medical,7336.548132,36 months
8,moving,6678.383741,36 months
9,other,8163.823719,36 months


### **CAST Function in SQL**

The `CAST` function in SQL is used to explicitly convert a value or expression from one data type to another. It is particularly useful when you need to change the data type of a column or an expression in your SQL query. 




In [37]:
pd.read_sql_query("""
    SELECT purpose, MAX(CAST(loan_amount AS integer)) AS MaxDisbursed
    FROM record
    GROUP BY purpose
    ORDER BY MaxDisbursed DESC                                            
""", con=conn)

,purpose,MaxDisbursed
0,wedding,35000
1,vacation,35000
2,small_business,35000
3,renewable_energy,35000
4,other,35000
5,moving,35000
6,medical,35000
7,major_purchase,35000
8,house,35000
9,home_improvement,35000


In [42]:
pd.read_sql_query("""
    SELECT purpose,
           loan_condition,
           AVG(CAST(loan_amount AS integer)) AS AvgPrincipal, 
           AVG(CAST(interest_rate AS decimal)) AS AvgInterest
    FROM record
    WHERE income_category = 'Low'
    GROUP BY purpose, loan_condition
    ORDER BY purpose                                            
""", con=conn)

,purpose,loan_condition,AvgPrincipal,AvgInterest
0,car,Bad Loan,8746.620370,14.571000
1,car,Good Loan,8262.366685,12.002414
2,credit_card,Bad Loan,14111.840834,14.667090
3,credit_card,Good Loan,13803.955620,11.794627
4,debt_consolidation,Bad Loan,14492.433535,16.154171
5,debt_consolidation,Good Loan,14037.003174,13.537448
6,educational,Bad Loan,6615.312500,13.398750
7,educational,Good Loan,5883.637874,11.842359
8,home_improvement,Bad Loan,13827.229130,15.543670
9,home_improvement,Good Loan,12326.139271,13.058086


## **HAVING Clause in SQL**

The `HAVING` clause in SQL is used to filter the results of a `GROUP BY` query based on a specified condition. It allows you to apply a condition to groups of rows, similar to how the `WHERE` clause filters individual rows. However, the `HAVING` clause operates on aggregated data, typically when using aggregate functions like `SUM`, `AVG`, `COUNT`, or `MAX` in conjunction with the `GROUP BY` clause.

The basic syntax of the `HAVING` clause is as follows:

```sql
SELECT column1, column2, ...
FROM table
GROUP BY column1, column2, ...
HAVING condition;


In [47]:
pd.read_sql_query("""
    SELECT region, COUNT(region) AS total
    FROM record
    GROUP BY region
    HAVING total > 200000
""",con=conn)

,region,total
0,Northern-Irl,204399
1,leinster,214646
2,ulster,208731


In [52]:
pd.read_sql_query("""
    SELECT purpose, loan_condition, AVG(CAST(loan_amount AS integer)) AS AvgPrincipal, AVG(CAST(interest_rate AS decimal)) AS AvgInterest
    FROM record
    WHERE income_category = 'Low'
    GROUP BY purpose, loan_condition
    HAVING AvgPrincipal > 10000
    ORDER BY purpose
""",con=conn)

,purpose,loan_condition,AvgPrincipal,AvgInterest
0,credit_card,Bad Loan,14111.840834,14.667090
1,credit_card,Good Loan,13803.955620,11.794627
2,debt_consolidation,Bad Loan,14492.433535,16.154171
3,debt_consolidation,Good Loan,14037.003174,13.537448
4,home_improvement,Bad Loan,13827.229130,15.543670
5,home_improvement,Good Loan,12326.139271,13.058086
6,house,Bad Loan,13510.526316,17.946371
7,house,Good Loan,12854.986231,15.863568
8,major_purchase,Bad Loan,11448.651079,15.406088
9,major_purchase,Good Loan,10130.592491,12.636636


## **Subquery**

A subquery in SQL, also known as an inner query or nested query, is a query that is embedded within another SQL query. Subqueries are a powerful feature that allows you to perform complex queries by breaking them down into smaller, more manageable parts. Subqueries can be used in various SQL statements, such as `SELECT`, `INSERT`, `UPDATE`, or `DELETE`.

The basic structure of a subquery is as follows:

```sql
SELECT column1, column2, ...
FROM table1
WHERE columnX operator (SELECT columnY FROM table2 WHERE condition);

In [54]:
pd.read_sql_query("""
    SELECT *
    FROM record
    WHERE region IN (
        SELECT region
        FROM record
        GROUP BY region
        HAVING COUNT(region) > 200000
    )
    LIMIT 20;              
""", con = conn)

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077430,2011,01/12/2011,1042013,0.5,RENT,1,Low,30000,1,...,1,15.27,C,3,1.0,1008.71,456.46,117.08,59.83,leinster
1,1076863,2011,01/12/2011,1012015,10.0,RENT,1,Low,49200,1,...,0,13.49,C,3,20.0,12226.30221,10000.0,0.0,339.31,ulster
2,1075358,2011,01/12/2011,1012016,1.0,RENT,1,Low,80000,1,...,0,12.69,B,2,17.94,3242.17,2233.1,0.0,67.79,ulster
3,1069639,2011,01/12/2011,1012016,8.0,RENT,1,Low,47004,1,...,0,15.96,C,3,23.51,8136.84,5110.85,0.0,170.08,leinster
4,1072053,2011,01/12/2011,1012015,9.0,RENT,1,Low,48000,1,...,0,18.64,E,5,5.35,3938.1443340000005,3000.0,0.0,109.43,ulster
5,1071795,2011,01/12/2011,1042012,4.0,OWN,2,Low,40000,1,...,1,21.28,F,6,5.55,646.02,162.02,189.06,152.39,ulster
6,1069908,2011,01/12/2011,1092013,10.0,OWN,2,Low,75000,1,...,0,12.69,B,2,10.78,13943.08,12000.0,0.0,402.54,ulster
7,1064687,2011,01/12/2011,1072012,0.5,RENT,1,Low,30000,1,...,1,13.49,C,3,10.08,2270.7,1256.14,444.3,305.38,leinster
8,1069057,2011,01/12/2011,1102013,3.0,RENT,1,Low,100000,1,...,1,10.65,B,2,7.06,7471.99,5433.47,645.1,325.74,ulster
9,1065775,2011,01/12/2011,1012015,4.0,RENT,1,Low,42000,1,...,0,15.27,C,3,18.6,12519.26045,10000.0,0.0,347.98,ulster


### **Between**

The `BETWEEN` operator in SQL is used to filter rows based on a range of values for a specified column. It allows you to select rows where a column's value falls within a specified range, inclusive of both endpoints.

The basic syntax of the `BETWEEN` operator is as follows:

```sql
SELECT column1, column2, ...
FROM table
WHERE columnX BETWEEN value1 AND value2;

In [57]:
pd.read_sql_query("""
    SELECT * 
    FROM record
    WHERE region IN (
        SELECT region
        FROM record
        GROUP BY region
        HAVING COUNT(region) BETWEEN 200000 and 210000
    ) AND
        CAST(loan_amount AS integer) BETWEEN 15000 AND 35000
    LIMIT 20;
""", con=conn)

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1069800,2011,01/12/2011,1012016,9.0,RENT,1,Low,60000,1,...,1,14.27,C,3,15.22,0.0,0.0,0.0,514.64,Northern-Irl
1,1069539,2011,01/12/2011,1082013,5.0,MORTGAGE,3,Low,75000,1,...,0,7.9,A,1,14.03,34886.08,31825.0,0.0,995.82,Northern-Irl
2,1068120,2011,01/12/2011,1012015,9.0,RENT,1,Medium,110000,2,...,0,9.91,B,2,15.71,29695.62268,25600.0,0.0,824.96,ulster
3,1069410,2011,01/12/2011,1102014,7.0,RENT,1,Low,50000,1,...,1,19.91,E,5,21.58,18319.14,8990.81,0.0,555.33,ulster
4,1068409,2011,01/12/2011,1102013,7.0,RENT,1,Low,81000,1,...,0,19.91,E,5,20.52,20876.1,16000.0,0.0,423.11,Northern-Irl
5,1068934,2011,01/12/2011,1012016,6.0,RENT,1,Low,40000,1,...,0,17.27,D,4,19.47,10713.52,6487.75,0.0,223.74,ulster
6,1069030,2011,01/12/2011,1112012,4.0,RENT,1,Low,44544,1,...,0,14.27,C,3,22.71,18172.71,16425.0,0.0,563.53,ulster
7,1068994,2011,01/12/2011,1022014,3.0,MORTGAGE,3,Medium,150000,2,...,0,17.27,D,4,7.51,28871.92,22075.0,0.0,551.84,Northern-Irl
8,1068542,2011,01/12/2011,1042013,7.0,MORTGAGE,3,Low,45000,1,...,1,17.27,D,4,21.31,10037.1,3077.85,3486.0,437.47,ulster
9,1068326,2011,01/12/2011,1012015,10.0,RENT,1,Low,45000,1,...,0,10.65,B,2,14.8,28140.929360000002,24000.0,0.0,781.76,ulster


### **View**

- masi error

In [ ]:
pd.read_sql_query("""
CREATE VIEW keymarket
AS
      SELECT id, region, loan_amount
      FROM record
      WHERE region IN(
        SELECT region
        FROM record
        GROUP BY region
        HAVING COUNT(region) BETWEEN 200000 AND 210000
    ) AND 
        CAST(loan_amount AS integer) BETWEEN 15000 AND 35000
    LIMIT 20;
""",con=conn)

## rand

In [10]:
df.dtypes

id                      object
year                    object
issue_d                 object
final_d                 object
emp_length_int          object
home_ownership          object
home_ownership_cat      object
income_category         object
annual_inc              object
income_cat              object
loan_amount             object
term                    object
term_cat                object
application_type        object
application_type_cat    object
purpose                 object
purpose_cat             object
interest_payments       object
interest_payment_cat    object
loan_condition          object
loan_condition_cat      object
interest_rate           object
grade                   object
grade_cat               object
dti                     object
total_pymnt             object
total_rec_prncp         object
recoveries              object
installment             object
region                  object
dtype: object

In [13]:
unique_values = df[['region', 'home_ownership_cat', 'year']].values.ravel('K')
unique_values = pd.unique(unique_values)

print(unique_values)


['munster' 'leinster' 'cannught' 'ulster' 'Northern-Irl' '1' '2' '3' '4'
 '5' '6' '2011' '2010' '2009' '2008' '2007' '2013' '2012' '2014' '2015']


In [14]:
import numpy as np

unique_values = np.ravel(df[['region', 'home_ownership_cat', 'year']].values)
unique_values = np.unique(unique_values)

print(unique_values)


['1' '2' '2007' '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015'
 '3' '4' '5' '6' 'Northern-Irl' 'cannught' 'leinster' 'munster' 'ulster']


In [18]:
unique_values = df[['region', 'home_ownership_cat', 'year']].values.ravel('F')
unique_values = pd.unique(unique_values)

print(unique_values)


['munster' 'leinster' 'cannught' 'ulster' 'Northern-Irl' '1' '2' '3' '4'
 '5' '6' '2011' '2010' '2009' '2008' '2007' '2013' '2012' '2014' '2015']
